In [2]:
file_prefix <- "ch20.1"
reuse_fit <- T
source("preset.R")

input_file <- "data/Salary.csv"

# prepare data

In [34]:
data <- read_csv(input_file) %>% 
    mutate(Pos = factor(Pos, levels=c("FT3","FT2","FT1","NDW","DST"), ordered=TRUE, 
                        labels=c("Assis","Assoc","Full","Endow","Disting")), 
           Org = factor(Org))

Parsed with column specification:
cols(
  Org = col_character(),
  OrgName = col_character(),
  Cla = col_character(),
  Pos = col_character(),
  ClaPos = col_character(),
  Salary = col_double()
)



In [35]:
head(data)

Org,OrgName,Cla,Pos,ClaPos,Salary
<fct>,<chr>,<chr>,<ord>,<chr>,<dbl>
PL,Philosophy,PC,Assoc,PC.FT2,72395
MUTH,Music Theory,PC,Assoc,PC.FT2,61017
ENG,English,PC,Assoc,PC.FT2,82370
CMLT,Comparative Literature,PC,Assoc,PC.FT2,68805
LGED,Language Education,PT,Assis,PT.FT3,63796
MGMT,Dept Mangement & Entrep,PR,Endow,PR.NDW,219600


In [30]:
y_name = "Salary"
x1_name = "Pos"
x2_name = "Org"

In [37]:
y_M = mean(data[[y_name]])
y_SD = sd(data[[y_name]])

In [38]:
stan_data <- list(
    N = nrow(data), 
    K1 = n_distinct(data[x1_name]), 
    K2 = n_distinct(data[x2_name]), 
    x1 = as.integer(data[[x1_name]]), 
    x2 = as.integer(data[[x2_name]]), 
    y = data[[y_name]], 
    y_M = y_M, 
    y_SD = y_SD,
    a_gamma_SH_RA = unlist(gammaShRaFromModeSD(y_SD / 2, y_SD * 2))
)

In [57]:
stan_data$y_SD

[1] 44283.2

# model fit 

In [7]:
if (reuse_fit) {
    load(save_file_name("fit"))
} else {
    fit <- stan(file = "ch20.1.stan", data = stan_data, seed = 123, 
            pars = c("m", "alpha_0", "alpha1", "alpha2"), include = F, 
            chains = 4, warmup = 300, iter = 10000, thin = 4)
    save(model, fit, file = save_file_name("fit"))
}

recompiling to avoid crashing R session



In [83]:
res <- fit_to_tibble(fit) %>% 
    select("beta_0", "beta1_1", "beta1_2", "beta1_3", "beta1_4", "beta1_5", 
           "beta2_21", "beta2_49", "beta2_13", "beta2_8", 
           "beta12_1,49", "beta12_3,49", "beta12_1,13", "beta12_3,13",
           "y_sigma")

In [87]:
res %>% summarise_all(funs(mean)) %>% 
    gather(key, value, beta_0:y_sigma)

key,value
<chr>,<dbl>
beta_0,129950.353
beta1_1,-46780.660
beta1_2,-36101.757
beta1_3,-6883.508
beta1_4,33393.004
beta1_5,56372.920
beta2_21,-11401.590
beta2_49,2004.458
beta2_13,11183.380
